In [6]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# Estrategias de Optimización de Velocidad

Los codigos enfocados a la eficiencia de velocidad de procesamiento implementan varias estrategias para procesar eficientemente grandes volúmenes de datos con un uso mínimo de memoria RAM. A continuación, se detallan estas estrategias:

## 1. Uso de `ujson`
- **Implementación**: `import ujson`
- **Beneficio**: `ujson` es significativamente más rápido que el módulo `json` estándar para parsear JSON.

## 2. Carga Completa en Memoria
- **Técnica**: El archivo JSON completo se carga en memoria utilizando `file.readlines()`.
- **Beneficio**: Esto minimiza el tiempo de I/O (entrada/salida) del disco, que suele ser un cuello de botella. Una vez que el archivo está en memoria, todo el procesamiento se realiza en la RAM, que es mucho más rápida.

## 3. Paralelización Completa
- **Explicación**: El código utiliza multiprocessing.Pool para distribuir el procesamiento de líneas entre múltiples núcleos de CPU disponibles. `num_processes = mp.cpu_count() ` garantiza que se aproveche al máximo el hardware disponible.
- **Beneficio**: Esto permite que múltiples líneas se procesen simultáneamente, lo que reduce significativamente el tiempo total de ejecución en comparación con el procesamiento secuencial.

## 4. Extracción directa de datos
- **Implementación**: `date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).date()`
- **Ventaja**: Conversión rápida y directa de la fecha sin procesamiento adicional.

## 5. Uso eficiente de `heapq.nlargest`
- **Código**: `top_10_dates = heapq.nlargest(10, date_tweet_counts.items(), key=lambda x: x[1])`
- **Eficiencia**: Encuentra los top 10 en tiempo O(n log k), donde k = 10, más rápido que ordenar toda la lista.



In [7]:
import cProfile
import pstats
import io

def profile_function(func, file_path):
    pr = cProfile.Profile()
    pr.enable()
    
    result = func(file_path)
    
    pr.disable()
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()
    
    print(s.getvalue())
    print(f"Resultado (primeros 10): {result[:10]}")
    
    return result

In [10]:
from q1_time import q1_time
print("Perfilando q1_time:")
result_q1_time = profile_function(q1_time, file_path)


Perfilando q1_time:
         155753 function calls in 1.833 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.080    0.080    1.833    1.833 c:\Users\nicol\OneDrive\Escritorio\LatFirstCommmit\src\q1_time.py:17(q1_time)
        4    0.000    0.000    0.688    0.172 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:604(wait)
        4    0.000    0.000    0.688    0.172 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\threading.py:288(wait)
       19    0.688    0.036    0.688    0.036 {method 'acquire' of '_thread.lock' objects}
        1    0.000    0.000    0.688    0.688 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:362(map)
        1    0.000    0.000    0.687    0.687 c:\Users\nicol\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:767(get)
        1    0.000    0.000    0.687    0.687 c:\Users\nicol\AppData\Local\Progr

# Estrategias para Reducir el Consumo de Memoria RAM

Los codigos enfocados a la eficiencia de memoria implementan varias estrategias para procesar eficientemente grandes volúmenes de datos con un uso mínimo de memoria RAM. A continuación, se detallan estas estrategias:

## 1. Procesamiento línea por línea
- **Método**: Lee y procesa el archivo JSON línea por línea.
- **Beneficio**: Solo un tweet está en memoria a la vez, reduciendo significativamente el uso de RAM.

## 2. Estructuras de datos eficientes
- **Uso**: Emplea `Counter` y `defaultdict(Counter)` para contar.
- **Ventaja**: Más eficientes en memoria que diccionarios personalizados para conteo.

## 3. Heap para top 10
- **Implementación**: Mantiene un heap de tamaño fijo (10 elementos) para los top 10 días.
- **Eficiencia**: Evita ordenar toda la lista de días al final, ahorrando memoria.

## 4. Procesamiento incremental
- **Método**: Actualiza conteos y heap mientras lee el archivo.
- **Beneficio**: Evita almacenar todos los datos para procesarlos al final.

## 5. Uso de `ujson`
- **Característica**: Más eficiente en memoria que el módulo `json` estándar.

In [16]:
import ujson
from collections import Counter, defaultdict
from datetime import datetime
from typing import List, Tuple
import heapq
from memory_profiler import profile
from q1_memory import q1_memory

@profile
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_tweet_counts = Counter()
    date_user_counts = defaultdict(lambda: Counter())
    top_dates_heap = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            tweet = ujson.loads(line)
            date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).date()
            username = tweet['user']['username']
            
            date_tweet_counts[date] += 1
            date_user_counts[date][username] += 1
            
            if len(top_dates_heap) < 10:
                heapq.heappush(top_dates_heap, (date_tweet_counts[date], date))
            elif date_tweet_counts[date] > top_dates_heap[0][0]:
                heapq.heapreplace(top_dates_heap, (date_tweet_counts[date], date))
    
    result = []
    for _, date in sorted(top_dates_heap, reverse=True):
        top_user = date_user_counts[date].most_common(1)[0][0]
        result.append((date, top_user))
    
    return result


result = q1_memory( file_path)
print(f"Resultado (primeros 10): {result[:10]}")

Filename: C:\Users\nicol\AppData\Local\Temp\ipykernel_20028\3829862783.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     9    904.8 MiB    904.8 MiB           1   @profile
    10                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    11    904.8 MiB      0.0 MiB           1       date_tweet_counts = Counter()
    12    904.8 MiB      0.0 MiB          27       date_user_counts = defaultdict(lambda: Counter())
    13    904.8 MiB      0.0 MiB           1       top_dates_heap = []
    14                                             
    15    904.9 MiB      0.0 MiB           2       with open(file_path, 'r', encoding='utf-8') as file:
    16    904.9 MiB      0.0 MiB      117408           for line in file:
    17    904.9 MiB      0.0 MiB      117407               tweet = ujson.loads(line)
    18    904.9 MiB      0.0 MiB      117407               date = datetime.fromisoformat(tweet['date'].replace('Z', '+00:00')).